# New Data Analysis

In [1]:
dataDir='Data/'

In [2]:
import json
import pandas as pd
import numpy as np
import re

## Openning the data¶

In [3]:
#function that open the file and create a "good" dataset
def createDataFrame(file):
    ##lists of 50 data are loaded and then merged
    #create list of dataframes and open file
    dataframes=[]
    file=open(dataDir+file, 'r')
    
    #counter is important for the index
    c=0
    while True:
        #each line is 50 data (as limit=50 when downloading)
        txt=file.readline()
        #possibly stop
        if txt=='':
            break

        #load the data with right index
        j=json.loads(txt)
        l=len(j)
        dataframes.append(pd.DataFrame(j, index=list(range(c, c+l))))
        
        c+=l
    #merge all datasets and return result
    return pd.concat(dataframes)

In [39]:
dfEvent=createDataFrame('event.json')

In [5]:
dfEvent.columns

Index(['beneficiaries_id', 'canceled', 'checks_payable_to_address',
       'contributions_info', 'distribution_paid_for_by', 'end_date',
       'end_time', 'entertainment', 'hosts_id', 'id', 'is_presidential',
       'make_checks_payable_to', 'more_details', 'notes', 'other_members',
       'postponed', 'resource_uri', 'rsvp_info', 'start_date', 'start_time',
       'venue_id'],
      dtype='object')

In [6]:
dfEvent

,beneficiaries_id,canceled,checks_payable_to_address,contributions_info,distribution_paid_for_by,end_date,end_time,entertainment,hosts_id,id,...,make_checks_payable_to,more_details,notes,other_members,postponed,resource_uri,rsvp_info,start_date,start_time,venue_id
0,[125],False,"617 East Curtis Avenue, Alexandria, VA 22301",,,None,None,Sushi Lunch Series Benefiting Citizens for Turner,[1793],42563,...,Citizens for Turner,,,[],False,/api/v1/event/42563/,mgroup2000@verizon.net,2017-12-07,12:00:00,2148
1,[136],False,"217 Third Street SE, Washington, DC 20003",,,None,None,2017 Royce Breakfast Series,[],42564,...,Royce Campaign Committee,,,[],False,/api/v1/event/42564/,maggie@bogartassociates.com,2017-12-07,08:30:00,2675
2,[19816],False,"47 Flintlock Drive, Shirley, NY 11967",,,None,None,New York Bagel Breakfast,[],42562,...,Zeldin for Congress,,,[],False,/api/v1/event/42562/,rsvp@aurumconsultingdc.com,2017-11-30,09:00:00,4818
3,"[42, 17513]",False,"Suite 422, 499 South Capitol Street SW, Washin...",,DelBene for Congress,2017-09-14,13:30:00,Luncheon,[6362],42804,...,DelBene for Congress,,,[],False,/api/v1/event/42804/,Patrick White or Lindsay Angerholzer at (202) ...,2017-11-14,12:30:00,15
4,[125],False,"617 East Curtis Avenue, Alexandria, VA 22301",,,None,None,Sushi Lunch Series Benefiting Citizens for Turner,[],42561,...,Citizens for Turner,,,[],False,/api/v1/event/42561/,mgroup2000@verizon.net,2017-11-14,12:00:00,2148
5,[18581],False,"4509 Saucon Valley Court, Alexandria, VA, 22312",,,2017-11-04,None,"Senator Jeff and Cheryl Flake's ""Fun in the Su...",[],42711,...,Jeff Flake for US Senate c/o Prescott Strategies,,,[],False,/api/v1/event/42711/,meredith@prescottstrategies.com,2017-11-03,None,6303
6,[136],False,"217 Third Street SE, Washington, DC 20003",,,None,None,2017 Royce Breakfast Series,[],42560,...,Royce Campaign Committee,,,[],False,/api/v1/event/42560/,maggie@bogartassociates.com,2017-11-02,08:30:00,2675
7,[19816],False,"47 Flintlock Drive, Shirley, NY 11967",,,None,None,New York Bagel Breakfast,[],42559,...,Zeldin for Congress,,,[],False,/api/v1/event/42559/,Kerry Donnelly rsvp@aurumconsultingdc.com 917-...,2017-10-26,09:00:00,4818
8,[17256],False,"3410 Alabama Avenue, Alexandria, VA 22305",,,None,None,Kentucky Weekend at Keeneland,[],42589,...,Building America's Republican Representation PAC,,,[],False,/api/v1/event/42589/,ali@stantongrpllc.com,2017-10-20,None,NaN
9,[18510],False,,,,2017-10-17,14:00:00,,[],42806,...,Collins Victory Fund,,,[],False,/api/v1/event/42806/,Kristine Bieniek 716-391-0052,2017-10-17,12:00:00,NaN


--
### Number of events:

In [7]:
#number of tracked events:
len(dfEvent)

24108

--
### Contributions for each event:

#### looking for host contribution:

see https://regex101.com/r/pj3UC4/1 for more info on the reg expr

In [40]:
def getHostContrib(s):
    reg_expr=re.compile(r'((h|H)ost[ Commitee\:]+\$?[,0-9]+|\$?[,0-9]+[ per\-PACto]+(h|H)ost( [^$]|;|\n|$)|(h|H)ost - \$[,0-9]+)')
    reg_expr2=re.compile(r"[0-9,][0-9,]+")

    if s is None:
        #print(None)
        return None
    else:
        hosts = re.finditer(reg_expr, s)
        c=0
        for hostNum, host in enumerate(hosts):
            c+=1
            a=host.group(0)
        if c==1:
            s=a
        
        n=[]
        nbs = re.finditer(reg_expr2, s)
        for nbNum, nb in enumerate(nbs):
            n.append(int(nb.group(0).replace(',', '')))
        try:
            #print(int(max(n)))
            return max(n)
        except:
            return None
        
        
print(getHostContrib("Host: $2,500 - PAC | $1,500 - Individual / Att..."))
print(getHostContrib("PAC Host | $2,500 - Individual Host /$500 - Individual Guest"))


2500
2500


In [41]:
dfEvent['contributions_info']=dfEvent['contributions_info'].apply(getHostContrib)

In [43]:
dfEvent['contributions_info'].count()

20033

20033 values for contribution over the 24108 events

In [44]:
dfEvent

,beneficiaries_id,canceled,checks_payable_to_address,contributions_info,distribution_paid_for_by,end_date,end_time,entertainment,hosts_id,id,...,make_checks_payable_to,more_details,notes,other_members,postponed,resource_uri,rsvp_info,start_date,start_time,venue_id
0,[125],False,"617 East Curtis Avenue, Alexandria, VA 22301",NaN,,None,None,Sushi Lunch Series Benefiting Citizens for Turner,[1793],42563,...,Citizens for Turner,,,[],False,/api/v1/event/42563/,mgroup2000@verizon.net,2017-12-07,12:00:00,2148
1,[136],False,"217 Third Street SE, Washington, DC 20003",NaN,,None,None,2017 Royce Breakfast Series,[],42564,...,Royce Campaign Committee,,,[],False,/api/v1/event/42564/,maggie@bogartassociates.com,2017-12-07,08:30:00,2675
2,[19816],False,"47 Flintlock Drive, Shirley, NY 11967",NaN,,None,None,New York Bagel Breakfast,[],42562,...,Zeldin for Congress,,,[],False,/api/v1/event/42562/,rsvp@aurumconsultingdc.com,2017-11-30,09:00:00,4818
3,"[42, 17513]",False,"Suite 422, 499 South Capitol Street SW, Washin...",NaN,DelBene for Congress,2017-09-14,13:30:00,Luncheon,[6362],42804,...,DelBene for Congress,,,[],False,/api/v1/event/42804/,Patrick White or Lindsay Angerholzer at (202) ...,2017-11-14,12:30:00,15
4,[125],False,"617 East Curtis Avenue, Alexandria, VA 22301",NaN,,None,None,Sushi Lunch Series Benefiting Citizens for Turner,[],42561,...,Citizens for Turner,,,[],False,/api/v1/event/42561/,mgroup2000@verizon.net,2017-11-14,12:00:00,2148
5,[18581],False,"4509 Saucon Valley Court, Alexandria, VA, 22312",NaN,,2017-11-04,None,"Senator Jeff and Cheryl Flake's ""Fun in the Su...",[],42711,...,Jeff Flake for US Senate c/o Prescott Strategies,,,[],False,/api/v1/event/42711/,meredith@prescottstrategies.com,2017-11-03,None,6303
6,[136],False,"217 Third Street SE, Washington, DC 20003",NaN,,None,None,2017 Royce Breakfast Series,[],42560,...,Royce Campaign Committee,,,[],False,/api/v1/event/42560/,maggie@bogartassociates.com,2017-11-02,08:30:00,2675
7,[19816],False,"47 Flintlock Drive, Shirley, NY 11967",NaN,,None,None,New York Bagel Breakfast,[],42559,...,Zeldin for Congress,,,[],False,/api/v1/event/42559/,Kerry Donnelly rsvp@aurumconsultingdc.com 917-...,2017-10-26,09:00:00,4818
8,[17256],False,"3410 Alabama Avenue, Alexandria, VA 22305",NaN,,None,None,Kentucky Weekend at Keeneland,[],42589,...,Building America's Republican Representation PAC,,,[],False,/api/v1/event/42589/,ali@stantongrpllc.com,2017-10-20,None,NaN
9,[18510],False,,NaN,,2017-10-17,14:00:00,,[],42806,...,Collins Victory Fund,,,[],False,/api/v1/event/42806/,Kristine Bieniek 716-391-0052,2017-10-17,12:00:00,NaN


NaN is when there is no value found

# Open secret data

In [13]:
dfContrib=createDataFrame("candContrib.json")
dfContrib.columns

Index(['cid', 'indivs', 'org_name', 'pacs', 'total', 'year'], dtype='object')

In [14]:
dfContrib.total=dfContrib.total.apply(int)#.sort_index()#'total')

In [15]:
dfContrib.sort_values(by='total', ascending=False)

,cid,indivs,org_name,pacs,total,year
380,N00035267,154044,NorPAC,4827,158871,2014
400,N00035267,154044,NorPAC,4827,158871,2018
390,N00035267,154044,NorPAC,4827,158871,2016
401,N00035267,156910,"Paul, Weiss et al",0,156910,2018
391,N00035267,154150,"Paul, Weiss Et Al",0,154150,2016
381,N00035267,153900,"Paul, Weiss et al",0,153900,2014
402,N00035267,100950,Gibbons PC,10000,110950,2018
392,N00035267,100950,Gibbons PC,10000,110950,2016
350,N00000699,106657,Lowenstein Sandler LLP,0,106657,2014
360,N00000699,85550,NorPAC,5000,90550,2016


In [16]:
grouped=dfContrib['total'].groupby(dfContrib['year'])
grouped

In [17]:
grouped.describe(include='all')

year       
2014  count       130.000000
      mean      23400.784615
      std       27602.586754
      min        5000.000000
      25%       10000.000000
      50%       10999.000000
      75%       20845.250000
      max      158871.000000
2016  count       140.000000
      mean      24105.957143
      std       26270.845031
      min        8000.000000
      25%       10000.000000
      50%       13700.000000
      75%       20843.500000
      max      158871.000000
2018  count       140.000000
      mean      17938.892857
      std       26264.276344
      min        2500.000000
      25%        5400.000000
      50%        9200.000000
      75%       15000.000000
      max      158871.000000
Name: total, dtype: float64